Домашнее задание к лекции "Основы веб-скрапинга и работы с API"
Задание 1.
Обязательная часть
Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = ['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>

Дополнительная часть (необязательная)
Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст_статьи>

In [38]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
KEYWORDS = ['python', 'парсинг']
URL = 'https://habr.com/ru/search/?'


In [42]:
post_times = []
post_titles = []
post_links = []
post_body = []
for keyword in KEYWORDS:
    params = {
        'q':keyword
    }
    req = requests.get(URL, params)
    soup = BeautifulSoup(req.text, 'html.parser')
    posts = soup.find_all('article', class_='post')
    for post in posts:
        post_times.append(post.find('span', class_='post__time').text)
        post_titles.append(post.find('h2', class_='post__title').text)
        post_links.append(post.find('a', class_='btn').get('href'))
        req2 = requests.get(post.find('a', class_='btn').get('href'))
        content = BeautifulSoup(req2.text, 'html.parser')
        post_body.append(content.find('div', class_ = 'post__text'))
        break
post_titles = list(map(lambda x: x.strip(), post_titles))
articles = pd.DataFrame({'время':post_times, 'заголовок':post_titles, 'ссылка':post_links, 'текст': post_body})
articles

,время,заголовок,ссылка,текст
0,21 апреля 2020 в 18:35,"Вышел Python 2.7.18, последний релиз ветки Pyt...",https://habr.com/ru/news/t/498364/#habracut,"<!DOCTYPE html>\n<html lang=""ru"" class=""no-js""..."
1,3 апреля 2019 в 15:21,"Правда про парсинг сайтов, или «все интернет-м...",https://habr.com/ru/post/446488/#habracut,"<!DOCTYPE html>\n<html lang=""ru"" class=""no-js""..."


In [52]:
req = requests.get('https://habr.com/ru/news/t/498364/#habracut')
content = BeautifulSoup(req.text, 'html.parser')
core = content.find('div', class_ = 'post__text')
core.text.strip()

'20 апреля 2020 года, спустя почти десять лет после выхода Python 2.7, выпущен «на волю» Python версии 2.7.18. Это последний релиз ветки Python 2.x, поддерживаемый параллельно с веткой 3.x. \n\r\nСо вчерашнего дня и далее ветка Python 2.х обновляться больше не будет. Всем разработчикам и пользователям рекомендуется перейти на Python 3.\n\r\nPython 2.7 вышел 3 июля 2010, а релиз Python 3.0 состоялся 3 декабря 2008 года. Изначально в организации Python Software Foundation планировали, что ветка 3.х в течение 5 лет получит должное распространение. Но этого не произошло из-за продолжения поддержки версии 2.7 и отсутствия у разработчиков интереса к ветке Python 3, которая не содержала кардинальных прорывных улучшений. \n\r\nПоэтому автор языка и руководитель проекта внес изменения в регламент поддержки Python 2.7, продлив жизненный цикл ветки 2.х до 10 лет сделав «Extend Python 2.7 life till 2020», причем с пометкой: «There will be no Python 2.8». \n\r\nИ вот, это дата настала. История Pyth

Задание 2.
Обязательная часть
Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:
EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>

Подсказка: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.

Дополнительная часть (необязательная)
Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.
Документация к API VK: https://vk.com/dev/methods , вам поможет метод wall.get

GROUP = 'netology'  
TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ
В итоге должен формироваться датафрейм со столбцами: <дата поста> - <текст поста>